<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>


# GitHub - Github Add New Github Issues As Page In Notion Database
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/GitHub/Github_Add_New_Github_Issues_As_Page_In_Notion_Database.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a>

**Tags:** #github #notion #operations #automation


**Author:** [Sanjeet Attili](https://linkedin.com/in/sanjeet-attili-760bab190/)

This notebook is used to create Notion page in a database when a new Github issue is created.
<br/>References :
- Github SDK: [https://github.com/PyGithub/PyGithub](https://github.com/PyGithub/PyGithub)

## Input


### Import libraries


In [ ]:
import naas
from naas_drivers import notion, github

### Setup Notion


<a href='https://docs.naas.ai/drivers/notion'>How to get your Notion integration token ?</a>

In [ ]:
# Notion token
notion_token = "..."

# Specify database id
notion_database_id = "..."

notion_client = notion.connect(notion_token)

### Setup Github


In [ ]:
# Github token
github_token = "..."

# Github repo on which we want to create issues.
github_repo_name = "..."

gh_client = github.connect(github_token)

### Setup Naas


In [ ]:
# Schedule your notebook every 15 minutes.
naas.scheduler.add(cron="*/15 * * * *")

#-> Uncomment the line below to remove your scheduler
# naas.scheduler.delete()

## Model


### Query notion database


In [ ]:
db_notion = notion_client.database.get(notion_database_id)
df_notion = db_notion.df()

for idx, row in df_notion.iterrows():
    if row.Name=='':
        df_notion.drop(index= idx, inplace=True)
    try:
        df_notion.drop(columns=['Tags'], inplace=True)
    except KeyError:
        pass

df_notion

### Get Github issues


In [ ]:
df_issues = gh_client.repos.get_issues(github_repo_name)
df_issues.head()

### Create notion page from Github issue


In [ ]:
def notion_page_from_gh_issue(issue, notion_database_id):
    
    if len(df_notion) == 0:
        pass
    elif len(df_notion) != 0 and str(issue.issue_id) in df_notion.Issue_id.to_list():
        return "issue already exists in database"
    
    page = notion_client.page.create(database_id=notion_database_id, title = issue.issue_title)
    page.rich_text("URL",issue.link_to_the_issue)
    page.rich_text("Assignees",issue.issue_assignees)
    page.number('Issue_id',issue.issue_id)
    page.date('Last_created',str(issue.last_created_date)+'T'+str(issue.last_created_time))
    page.update()
    return "Done"

## Output


### Iterate over each issue.


In [ ]:
for idx, issue in df_issues.iterrows():
    val = notion_page_from_gh_issue(issue, notion_database_id)
    if val == "issue already exists in database":
        print("Database up to date!")
        break
    print(f'✅ Notion page created for issue {issue.link_to_the_issue}')